In [ ]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai chromadb
!pip3 install --upgrade --quiet pypdf pandas streamlit python-dotenv

In [ ]:

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

import os
import tempfile
import streamlit as st
import pandas as pd
from dotenv import load_dotenv



In [ ]:

load_dotenv()


In [5]:

OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")


In [ ]:
#Define our LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
llm.invoke("Tell me a joke about cats please")

In [ ]:
## Process the PDF Document
loader = PyPDFLoader('data/Annie Jacobsen - Operation Paperclip. The Secret Intelligence Program that Brought Nazi Scientists to America - 2014.pdf')
pages= loader.load()
pages

In [ ]:
## Split Document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500,
                                               chunk_overlap=200,
                                               length_function=len,
                                               separators= ["\n\n", '\n', " "])
text_splitter.split_documents(pages)

In [ ]:

## Create Embeddings

def get_embedding_function():
    embeddings = OpenAIEmbeddings(
        model= "text-embedding-ada-002", openai_api_key=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()
test_vector= embedding_function.embed_query('cat')
from langchain.evaluation import load_evaluator

evaluator = load_evaluator(evaluator="embedding_distance",
                           embeddings = embedding_function)

evaluator.evaluate_strings(prediction = "Nazi", reference = "German")
evaluator.evaluate_strings(prediction = "Nazi", reference = "American")
## Create A Vector database

import uuid

def create_vectorstore(chunks, embedding_function, vectorstore_path):
    #create a list if unique ids for each document bsed on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in docs]

    #ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []

    unique_chunks = []
    for chunk, id in zip(chunks.ids):
         if id not in unique_ids:
              unique_ids.add(id)
              unique_chunks.append(chunk)

    
    # Create a New Chroma Database for the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks,
                                        ids = list(unique_ids),
                                        embedding=embedding_function,
                                        persist_directory=vectorstore_path)
    
    vectorstore.persist()

    return vectorstore

#create a vecotr store
vectorstore = create_vectorstore(chunks = chunks,
                                  embedding_function = embedding_function,
                                  vectorstore_path="vectorstore_chroma")

# Load vectorestore
vectorstore= Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

# Create retirever and get relevant chunks
retriever = vectorstore.as_retriever(search_type='similarity')
relevant_chunks= retriever.invoke('What is the title of the book?')
relevant_chunks
# Query for relavant data
#load vectorstore 
vectorstore = Chroma(persist_directoy = "vectorstore_chroma", embedding_function=embedding_function)